<div align="center">

  <a href="https://ultralytics.com/yolov5" target="_blank">
    <img width="1024", src="https://raw.githubusercontent.com/ultralytics/assets/main/yolov5/v70/splash.png"></a>

[中文](https://docs.ultralytics.com/zh/) | [한국어](https://docs.ultralytics.com/ko/) | [日本語](https://docs.ultralytics.com/ja/) | [Русский](https://docs.ultralytics.com/ru/) | [Deutsch](https://docs.ultralytics.com/de/) | [Français](https://docs.ultralytics.com/fr/) | [Español](https://docs.ultralytics.com/es/) | [Português](https://docs.ultralytics.com/pt/) | [العربية](https://docs.ultralytics.com/ar/)

  <a href="https://bit.ly/yolov5-paperspace-notebook"><img src="https://assets.paperspace.io/img/gradient-badge.svg" alt="Run on Gradient"></a>
  <a href="https://colab.research.google.com/github/ultralytics/yolov5/blob/master/tutorial.ipynb"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"></a>
  <a href="https://www.kaggle.com/models/ultralytics/yolov5"><img src="https://kaggle.com/static/images/open-in-kaggle.svg" alt="Open In Kaggle"></a>

This <a href="https://github.com/ultralytics/yolov5">YOLOv5</a> 🚀 notebook by <a href="https://ultralytics.com">Ultralytics</a> presents simple train, validate and predict examples to help start your AI adventure.<br>We hope that the resources in this notebook will help you get the most out of YOLOv5. Please browse the YOLOv5 <a href="https://docs.ultralytics.com/yolov5">Docs</a> for details, raise an issue on <a href="https://github.com/ultralytics/yolov5">GitHub</a> for support, and join our <a href="https://ultralytics.com/discord">Discord</a> community for questions and discussions!

</div>

# Setup

Clone GitHub [repository](https://github.com/ultralytics/yolov5), install [dependencies](https://github.com/ultralytics/yolov5/blob/master/requirements.txt) and check PyTorch and GPU.

In [2]:
!git clone https://github.com/ultralytics/yolov5  # clone
%cd yolov5
%pip install -qr requirements.txt comet_ml  # install

import torch
import utils
display = utils.notebook_init()  # checks

YOLOv5 🚀 v7.0-408-g3fee72b5 Python-3.11.11 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)


Setup complete ✅ (2 CPUs, 12.7 GB RAM, 41.3/112.6 GB disk)


In [3]:
from google.colab import drive
drive.mount('/content/drive')

# Verify files (replace with your path)
!ls "/content/drive/MyDrive/annotated_imgs"

Mounted at /content/drive
K1_msz.png  K3_msz.png	K4_msz.png  T9_msz.png	Z9_msz.png


In [6]:
from google.colab import drive
drive.mount('/content/drive')

import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
from skimage.segmentation import watershed
from skimage.feature import peak_local_max
from scipy import ndimage

# Create directories if they don't exist
!mkdir -p "/content/drive/MyDrive/dataset/precise_labels"
!mkdir -p "/content/drive/MyDrive/dataset/visualizations"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [1]:
# ========== MOUNT DRIVE & IMPORTS ==========
from google.colab import drive

import cv2
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
from skimage.segmentation import watershed
from skimage.feature import peak_local_max
from scipy import ndimage
import re

# ========== CONFIGURATION ==========
ANNOTATED_DIR = Path("/content/drive/MyDrive/annotated_imgs")
ORIGINAL_DIR = Path("/content/drive/MyDrive/dataset/originals")
OUTPUT_DIR = Path("/content/drive/MyDrive/dataset/precise_labels")
VISUAL_DIR = Path("/content/drive/MyDrive/dataset/visualizations")

# ========== SAFE SEGMENTATION PARAMETERS ==========
INITIAL_HUE_TOLERANCE = 15
INITIAL_SAT_TOLERANCE = 25
MIN_BACTERIA_AREA = 50
MAX_BACTERIA_AREA = 2000

# ========== BULLETPROOF CORE FUNCTIONS ==========
def safe_resize(img, target_width, target_height):
    """Guaranteed safe resize with coordinate preservation"""
    if img.size == 0:
        return np.zeros((target_height, target_width, 3), np.zeros((target_height, target_width), np.uint8)
    return cv2.resize(img, (target_width, target_height), interpolation=cv2.INTER_NEAREST)

def adaptive_region_growing(annot_img, orig_img):
    # Get original dimensions first
    orig_height, orig_width = orig_img.shape[:2]

    # Resize annotation to match original dimensions
    annot_img = safe_resize(annot_img, orig_width, orig_height)

    # Convert to HSV space
    hsv = cv2.cvtColor(orig_img, cv2.COLOR_BGR2HSV)
    annot_hsv = cv2.cvtColor(annot_img, cv2.COLOR_BGR2HSV)

    # Detect blue annotations with triple validation
    blue_mask = cv2.inRange(annot_hsv, np.array([90,70,70]), np.array([130,255,255]))
    blue_mask = cv2.morphologyEx(blue_mask, cv2.MORPH_CLOSE, np.ones((5,5), np.uint8))

    # Safe contour detection
    contours, _ = cv2.findContours(blue_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    annotations = []
    for cnt in contours:
        M = cv2.moments(cnt)
        if M["m00"] > 0:
            cX = np.clip(int(M["m10"]/M["m00"]), 0, orig_width-1)
            cY = np.clip(int(M["m01"]/M["m00"]), 0, orig_height-1)
            annotations.append((cX, cY))

    # Create background mask with validation
    _, bg_mask = cv2.threshold(hsv[:,:,2], 0, 255, cv2.THRESH_BINARY+cv2.THRESH_OTSU)
    bg_mask = (bg_mask == 0).astype(np.uint8)

    # Region growing with boundary armor
    final_mask = np.zeros_like(hsv[:,:,0], dtype=np.uint8)
    for seed in annotations:
        x, y = np.clip(seed[0], 0, orig_width-1), np.clip(seed[1], 0, orig_height-1)
        best_seed = relocate_seed((x, y), hsv, bg_mask, (orig_width, orig_height))

        # Guard against invalid seeds
        if not (0 <= best_seed[0] < orig_width and 0 <= best_seed[1] < orig_height):
            continue

        # Region growing core
        h_ref, s_ref, _ = hsv[best_seed[1], best_seed[0]]
        region = np.zeros_like(final_mask)
        queue = [best_seed]
        visited = set()

        while queue:
            x, y = queue.pop(0)
            x = np.clip(x, 0, orig_width-1)
            y = np.clip(y, 0, orig_height-1)

            if (x, y) in visited: continue
            if bg_mask[y, x]: continue

            # Color similarity check with safety
            h, s, _ = hsv[y, x]
            if abs(int(h)-h_ref) < INITIAL_HUE_TOLERANCE and abs(int(s)-s_ref) < INITIAL_SAT_TOLERANCE:
                region[y, x] = 1
                visited.add((x, y))

                # Neighbor expansion with boundary armor
                for dx in [-1, 0, 1]:
                    for dy in [-1, 0, 1]:
                        nx = np.clip(x + dx, 0, orig_width-1)
                        ny = np.clip(y + dy, 0, orig_height-1)
                        queue.append((nx, ny))

        final_mask = cv2.bitwise_or(final_mask, region)

    # Post-processing fortress
    final_mask = cv2.morphologyEx(final_mask, cv2.MORPH_CLOSE, cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (7,7)))

    # Watershed with coordinate validation
    distance = ndimage.distance_transform_edt(final_mask)
    coords = peak_local_max(distance, min_distance=20, labels=final_mask)

    # Validate coordinates before marker creation
    valid_coords = []
    for y, x in coords:
        if 0 <= x < orig_width and 0 <= y < orig_height:
            valid_coords.append((x, y))

    markers = np.zeros_like(final_mask, dtype=np.int32)
    for i, (x, y) in enumerate(valid_coords):
        markers[y, x] = i + 1

    labels = watershed(-distance, markers, mask=final_mask)

    # Extract contours with boundary checks
    bacteria_contours = []
    for label in np.unique(labels):
        if label == 0: continue

        mask = np.zeros_like(labels, dtype=np.uint8)
        mask[labels == label] = 255

        cnts, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        for cnt in cnts:
            area = cv2.contourArea(cnt)
            if MIN_BACTERIA_AREA < area < MAX_BACTERIA_AREA:
                bacteria_contours.append(cnt)

    return bacteria_contours

def relocate_seed(original_seed, hsv_img, bg_mask, img_size):
    orig_width, orig_height = img_size
    x, y = np.clip(original_seed[0], 0, orig_width-1), np.clip(original_seed[1], 0, orig_height-1)
    best_score = -np.inf
    best_pos = (x, y)

    for dx in range(-7, 8):
        for dy in range(-7, 8):
            nx = np.clip(x + dx, 0, orig_width-1)
            ny = np.clip(y + dy, 0, orig_height-1)

            if bg_mask[ny, nx]: continue

            # Safe window extraction
            y1, y2 = max(0, ny-5), min(ny+6, orig_height)
            x1, x2 = max(0, nx-5), min(nx+6, orig_width)
            window = hsv_img[y1:y2, x1:x2]

            if window.size == 0: continue

            std_dev = np.std(window, axis=(0,1))
            score = std_dev[0] + std_dev[1]

            if score > best_score:
                best_score = score
                best_pos = (nx, ny)

    return best_pos

def process_image_pair(annot_path, orig_path):
    # Load images
    annot_img = cv2.imread(str(annot_path))
    orig_img = cv2.imread(str(orig_path))

    if annot_img is None or orig_img is None:
        print(f"⚠️ Error loading {annot_path.name} or {orig_path.name}")
        return

    # Perform segmentation
    contours = adaptive_region_growing(annot_img, orig_img)

    # Create visualization
    display_img = orig_img.copy()

    # Draw filled contours with transparency
    overlay = display_img.copy()
    cv2.drawContours(overlay, contours, -1, (0,255,0), -1)  # Green fill
    cv2.addWeighted(overlay, 0.3, display_img, 0.7, 0, display_img)

    # Draw contour edges
    cv2.drawContours(display_img, contours, -1, (255,0,0), 2)  # Blue edges

    # Create comparison figure
    fig, ax = plt.subplots(1, 3, figsize=(20, 6))
    fig.suptitle(f"Analysis for {orig_path.name}", fontsize=16, y=0.95)

    # Original Image
    ax[0].imshow(cv2.cvtColor(orig_img, cv2.COLOR_BGR2RGB))
    ax[0].set_title("Original Image\n", fontsize=12)
    ax[0].axis('off')

    # Annotation Points
    blue_mask = cv2.inRange(cv2.cvtColor(annot_img, cv2.COLOR_BGR2HSV),
                           np.array([90,70,70]), np.array([130,255,255]))
    ax[1].imshow(blue_mask, cmap='Blues')
    ax[1].set_title(f"Annotation Points: {len(contours)}\n", fontsize=12)
    ax[1].axis('off')

    # Segmentation Result
    ax[2].imshow(cv2.cvtColor(display_img, cv2.COLOR_BGR2RGB))
    ax[2].set_title(f"Segmented Bacteria: {len(contours)}\n", fontsize=12)
    ax[2].axis('off')

    plt.savefig(VISUAL_DIR / f"{orig_path.stem}_seg.png", bbox_inches='tight', dpi=150)
    plt.close()

    # Save labels
    save_yolo_labels(orig_img.shape, contours, orig_path.stem)

def save_yolo_labels(img_shape, contours, stem):
    height, width = img_shape[:2]
    with open(OUTPUT_DIR / f"{stem}.txt", "w") as f:
        for cnt in contours:
            # Get rotated bounding box
            rect = cv2.minAreaRect(cnt)
            box = cv2.boxPoints(rect)
            box_norm = box / np.array([width, height])

            # YOLO OBB format: class x1 y1 x2 y2 x3 y3 x4 y4
            line = f"0 " + " ".join([f"{p[0]:.6f} {p[1]:.6f}" for p in box_norm])
            f.write(line + "\n")

def find_matching_pairs(annotated_dir, original_dir):
    """Matches any files with pattern: <name>_msz.<ext> to <name>.<ext>"""
    pattern = re.compile(r'^(.+)_msz(\.[a-zA-Z]+)$', re.IGNORECASE)
    pairs = []

    for annot_path in annotated_dir.glob('*'):
        match = pattern.match(annot_path.name)
        if match:
            base_name = match.group(1) + match.group(2)
            orig_path = original_dir / base_name
            if orig_path.exists():
                pairs.append((annot_path, orig_path))
            else:
                print(f"⚠️ Missing original for {annot_path.name}")
    return pairs

def main():
    # Setup directories
    OUTPUT_DIR.mkdir(parents=True, exist_ok=True)
    VISUAL_DIR.mkdir(parents=True, exist_ok=True)

    # Find all valid image pairs
    pairs = find_matching_pairs(ANNOTATED_DIR, ORIGINAL_DIR)

    if not pairs:
        print("❌ No valid image pairs found!")
        print("Ensure annotated files follow <name>_msz.<ext> pattern")
        return

    processed = 0
    for annot_path, orig_path in pairs:
        try:
            process_image_pair(annot_path, orig_path)
            processed += 1
            print(f"✅ Processed {orig_path.name}")
        except Exception as e:
            print(f"❌ Error processing {orig_path.name}: {str(e)}")

    print(f"\n🎉 Finished! Processed {processed}/{len(pairs)} image pairs")
    print(f"Labels saved to: {OUTPUT_DIR}")
    print(f"Visualizations saved to: {VISUAL_DIR}")

if __name__ == "__main__":
    # Clear previous outputs (optional)
    !rm -rf "/content/drive/MyDrive/dataset/precise_labels/*"
    !rm -rf "/content/drive/MyDrive/dataset/visualizations/*"

    # Run the processing
    main()

Mounted at /content/drive
❌ Error processing K4.png: index 1774 is out of bounds for axis 0 with size 1536
❌ Error processing T9.png: index 1827 is out of bounds for axis 0 with size 1536
❌ Error processing K1.png: index 1652 is out of bounds for axis 0 with size 1536
❌ Error processing Z9.png: index 1682 is out of bounds for axis 0 with size 1536
❌ Error processing K3.png: index 1798 is out of bounds for axis 0 with size 1536

🎉 Finished! Processed 0/5 image pairs
Labels saved to: /content/drive/MyDrive/dataset/precise_labels
Visualizations saved to: /content/drive/MyDrive/dataset/visualizations
